# SedGen testing

## Imports

In [2]:
import numpy as np
import numba as nb
from numba.typed import List
import numexpr as ne
import pandas as pd
import itertools
from operator import itemgetter
import gc
import copy

from scipy.stats import norm, lognorm, truncnorm
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, deque
from sys import getsizeof

In [3]:
from sedgen.initialization import SedGen
from sedgen import initialization as ini
from sedgen import general as gen
from sedgen import preprocessing
from sedgen import geostatistics

In [4]:
%load_ext line_profiler
%load_ext memory_profiler

In [5]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time 
# before executing the Python code typed.
%autoreload 2

## Model input
___

### Parent rock characteristics

In [6]:
# Unit is mm³
parent_rock_volume = 1e9

# Mineral classes present in parent rock
minerals = ["Q", "P", "K", "B", "O", "A"]

### Modal mineralogy
This property will be represented by a mean value of parent rock samples.  
The mean value should be calculated based on clr transformed data, though.

In [7]:
# Modal mineralogy mean values f CA-NS of Heins(1992)
modal_mineralogy = np.array([0.30591989, 0.38159713, 0.26209888, 0.01882560, 0.00799247, 0.02356603])

### Interfacial composition
This property will be represented by a mean value of parent rock samples.  
The mean value should be calculated based on clr transformed data, though.

In [8]:
interface_proportions_true = None

### Crystal size distributions
This property will be based on a mean crystal size and standard deviation  
which will be represented as a log-normal distribution.

In [9]:
csds_CA_NS_means = np.array([0.309, 0.330, 0.244, 0.223, 0.120, 0.122])
csds_CA_NS_stds = np.array([0.823, 0.683, 0.817, 0.819, 0.554, 0.782])

## Initialization

In [16]:
# Create scenario data
balance = np.array([0.5] * 100)
climate = np.array([2] * 100)
# Keep additional input material limited to first couple of steps
# Otherwise this has a massive performance impact regarding the time it
# takes for the model to execute due to very high number of pcgs/mcgs.
new_input = np.array([0.01] * 10 + [0.0] * 90)
scenario_array = np.stack([balance, climate, new_input], axis=1)

In [17]:
%%time
sedgen_CA_NS = SedGen(minerals, 
                      parent_rock_volume, 
                      modal_mineralogy,
                      csds_CA_NS_means, 
                      csds_CA_NS_stds, 
                      scenario_data=scenario_array,
                      learning_rate=10000, 
                      timed=True,
                      chem_weath_rates=[0.01, 0.05, 0.1, 0.03, 0.02, 0.04])

---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A| Done in 3.0504 seconds
Initializing interfaces... |Q|P|K|B|O|A| Done in 2.8939 seconds
Counting interfaces...  Done in 0.0459 seconds
Correcting interface arrays for consistency...
too much Q 1
all good P 0
all good K 0
all good B 0
all good O 0
too few A -1
Initializing crystal size array... |Q|P|K|B|O|A| Done in 0.2289 seconds
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...
Initializing discretization for model's weathering...
100/100
---SedGen model initialization finished succesfully---
Wall time: 35 s


___

In [18]:
%%time
sedgen_CA_NS_mech = \
    sedgen_CA_NS.weathering(
        operations=["intra_cb", "inter_cb"], 
        timesteps=100, inplace=False)

After 78 steps all pcgs have been broken down to mcg
Wall time: 2min 10s


Examples of duration for new material input with only mechanical weathering:
- 10% new material every step out of 100 steps: 11m30s
- 10% new material for first 10 steps out of 100 steps: 4m58s
- 1% new material for first 10 steps out of 100 steps: 2m10s
- no new input: 1m58s

In [19]:
%%time
sedgen_CA_NS_chem = \
    sedgen_CA_NS.weathering(
        operations=["chem_mcg", "chem_pcg"], 
        timesteps=100, inplace=False)

Wall time: 37.8 s


Examples of duration for new material input with only chemical weathering:
- 10% new material every step out of 100 steps: not checked
- 10% new material for first 10 steps out of 100 steps: not checked
- 1% new material for first 10 steps out of 100 steps: 37.8s
- no new input: 24s

In [20]:
%%time
sedgen_CA_NS_both = \
    sedgen_CA_NS.weathering(
    operations=["intra_cb", "inter_cb", 
                "chem_mcg", "chem_pcg"], 
    timesteps=100, inplace=False)

After 74 steps all pcgs have been broken down to mcg
Wall time: 2min 19s


Examples of duration for new material input with both types of weathering:
- 10% new material every step out of 100 steps: not checked
- 10% new material for first 10 steps out of 100 steps: not checked
- 1% new material for first 10 steps out of 100 steps: 2m19s
- no new input: 1m06s

___